# A Monte Carlo Simulation Workflow in Python

## Introduction

This is a workflow for Monte Carlo Simulation in Python, using a dataframe to track the parameters, simulated dataset, and summary statistics / model fitting. For multi-core machines, we can try to speed up using parallel computation.

This workflow is inspired by R's `purrr` package and the notion of [list-columns](https://jennybc.github.io/purrr-tutorial/ls13_list-columns.html). I also benefited from Alex Hayes' [great note on many models workflow in python](https://www.alexpghayes.com/blog/many-models-workflows-in-python-part-i/).

**Running example in this note**:

1. Simulate many of datasets from each of the 9 normal distribution, with 3 different centers and 3 different scales.
2. Calculate the sample statistics for each of the simulated datasets.
3. Put everything in a dataframe.

**The workflow consists of 4 steps**:

1. Set the model parameters
2. Simulate datasets according to the parameters
3. Do stuff with the simulated datasets, e.g., calculate sample statistics, fit models.
4. Collect steps 1--3 in a dataframe. The main benefit of keeping everything in a dataframe is for later analysis and visualization of the Monte Carlo simulation.

In [1]:
from platform import python_version

print(python_version())

3.7.8


In [2]:
import concurrent.futures
from itertools import product
from typing import Dict, List, Union

import numpy as np
import pandas as pd

In [3]:
import psutil

n_cores = psutil.cpu_count()
n_cores

16

## Set model parameters

In [4]:
N_REP = 1000
# sample size can also be a parameter of simulation, but set to 1 value for illustration
SAMPLE_SIZE = [10_000]
locations = [0, 1, 2]
scales = [1, 2, 3]

params = pd.DataFrame(
    list(product(locations, scales, range(N_REP), SAMPLE_SIZE)),
    columns=["loc", "scale", "rep", "size"],
)
params

,loc,scale,rep,size
0,0,1,0,10000
1,0,1,1,10000
2,0,1,2,10000
3,0,1,3,10000
4,0,1,4,10000
...,...,...,...,...
8995,2,3,995,10000
8996,2,3,996,10000
8997,2,3,997,10000
8998,2,3,998,10000


## Simulate datasets

For small dataset, we can keep the simulated data sets in the tracking dataframe. If the simulated dataset is large, we only keep the sample statistics in the dataframe.

In [5]:
def simulate_data_seq(params: pd.DataFrame) -> pd.DataFrame:
    rng = np.random.default_rng()

    sim_data = pd.DataFrame()
    sim_data["sample"] = [
        rng.normal(loc=loc, scale=scale, size=size)
        for loc, scale, size in zip(params["loc"], params["scale"], params["size"])
    ]

    return sim_data

In [6]:
%%time
sim_data_seq = simulate_data_seq(params)

CPU times: user 1.23 s, sys: 156 ms, total: 1.38 s
Wall time: 1.38 s


## Do stuff with the simulated datasets

In [7]:
def calculate_sample_stats(arr: np.ndarray) -> Dict:
    """calculate sample statistics"""
    average = arr.mean()
    variance = arr.var(ddof=1)
    p05 = np.quantile(arr, 0.05)
    p95 = np.quantile(arr, 0.95)

    return {
        "average": average,
        "variance": variance,
        "p05": p05,
        "p95": p95,
    }


def compute_seq(df: pd.DataFrame) -> pd.DataFrame:
    sample_stats = [calculate_sample_stats(sample) for sample in df["sample"]]

    return pd.DataFrame.from_records(sample_stats)

In [8]:
%%time
sample_stats_seq = compute_seq(sim_data_seq)

CPU times: user 3.31 s, sys: 10.9 ms, total: 3.32 s
Wall time: 3.33 s


## Keep everything in a dataframe

In [9]:
pd.concat([params, sim_data_seq, sample_stats_seq], axis="columns")

,loc,scale,rep,size,sample,average,variance,p05,p95
0,0,1,0,10000,"[0.512323837070688, 1.6285988306617405, -0.360...",0.018938,0.992681,-1.618823,1.674992
1,0,1,1,10000,"[1.7358937856905425, 1.3084511253650128, -2.23...",0.006179,0.991738,-1.624149,1.652842
2,0,1,2,10000,"[-0.24762276193783542, 0.43955464441270564, 0....",-0.018337,0.996480,-1.655307,1.622204
3,0,1,3,10000,"[1.065171515045853, 0.17889437588697502, -0.02...",0.001312,0.976781,-1.608714,1.649646
4,0,1,4,10000,"[1.1429184613757573, -0.17184822288760265, 1.4...",-0.023902,0.993013,-1.664878,1.636161
...,...,...,...,...,...,...,...,...,...
8995,2,3,995,10000,"[1.1555760003586013, 10.904995345833228, 4.400...",2.006487,9.015386,-2.831194,6.951140
8996,2,3,996,10000,"[2.014605055383616, 0.368816960640737, 1.43875...",1.995852,9.070367,-2.893255,7.041648
8997,2,3,997,10000,"[1.059746906199416, 6.254749720859033, 4.80490...",2.023868,8.760861,-2.806729,6.912245
8998,2,3,998,10000,"[3.309197647210268, -0.3535167542851214, 1.779...",1.998481,8.960335,-2.981987,6.991106


## Parallel computation

We may be able to speed up the simulation and computation using parallel computation, e.g., multiprocessing, multithreading, or other packages, depending on whether the bottlenecks are the I/O, memory, cache, or CPU.

Below is an example of multiprocessing. A few notes when using multiprocessing in Python.

- Depending on the size of the datasets, number of datasets, and how cpu-hungry the computations are, multiprocessing may not perform better than sequential processing. In my working example, multiprocessing did not improve performance. In my real simulation work with heavier computations, multiprocessing does help.
- We need to change the simulation function a little in terms how it takes in arguments, to facilitate the `map` function.
- Chunksize may be a factor for performance.
- `Numpy`, `Scipy`, and `Pandas` scientific computing packages may not play well with multiprocessing. Read more [here](https://stackoverflow.com/a/15641148/3101585).

In [10]:
CHUNKSIZE = int(params.shape[0] / (n_cores / 2))
CHUNKSIZE

1125

In [11]:
def simulate_data(params: Dict) -> np.ndarray:
    loc = params["loc"]  # float
    scale = params["scale"]  # float
    size = params["size"]  # int

    # be sure to reseed the RNG
    # see discussions here: https://github.com/numpy/numpy/issues/9650
    rng = np.random.default_rng()

    return rng.normal(loc=loc, scale=scale, size=size)


def simulate_data_mp(params: List) -> Dict:

    params_list = params.to_dict(orient="records")
    sim_data = pd.DataFrame()

    with concurrent.futures.ProcessPoolExecutor() as executor:
        sim_data["sample"] = list(
            executor.map(simulate_data, params_list, chunksize=CHUNKSIZE)
        )

    return sim_data

In [12]:
%%time
sim_data_mp = simulate_data_mp(params)

CPU times: user 611 ms, sys: 4.35 s, total: 4.96 s
Wall time: 5.55 s


In [13]:
def compute_mp(df: pd.DataFrame) -> pd.DataFrame:
    with concurrent.futures.ProcessPoolExecutor() as executor:
        sample_stats = list(
            executor.map(calculate_sample_stats, df["sample"], chunksize=CHUNKSIZE)
        )

    return pd.DataFrame.from_records(sample_stats)

In [14]:
%%time
sample_stats_mp = compute_mp(sim_data_seq)

CPU times: user 442 ms, sys: 624 ms, total: 1.07 s
Wall time: 5.49 s
